# TESTEAR QUE A LAS RESTRICCIONES DEFINA MODELOS DE FORMA INDEPENDIENTES

# Gurobi optimization using one machine learning model
## Optimize for Price and Supply of Avocados

- In this example is only linear regressions, but gurobi machine learning acept multiple models

https://gurobi-machinelearning.readthedocs.io/en/stable/api.html


- In addition, to define the decision variables, parameters, restriction, etc of the optimization model are used "gurobipy-pandas". Using this package is possible define the optimization model using pandas DataFrames

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

### 0. Root repo

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

## PREPARATION

### 1. Load data needs to use
In this example data is loaded because it is necesary to generate parameters of optimization model

In [ ]:
# read data that have all the units sold for each region
path_data_basic_features = 'artifacts/data/data_basic_features.pkl'
data_units_sold = pd.read_pickle(path_data_basic_features)

In [ ]:
##### use data to generate parameters for optimization model

# min, max deliry each region
data_min_delivery = data_units_sold.groupby("region")["units_sold"].min().rename('min_delivery')
data_max_delivery = data_units_sold.groupby("region")["units_sold"].max().rename('max_delivery')

# historical distribution of price each region
data_historical_max_price = data_units_sold.groupby("region")["price"].max().rename('max_price')

### 2. Load model machine learning
Load model that given an input (price and other features) predict the price.

The model was trained in the notebook "models/1_basic_features_one_lr"

In [ ]:
# folder
path_folder_artifacts = 'artifacts/models/1_basic_features_one_lr/'

# path to save
name_artifact_model = 'model.pkl'
path_artifact_model = path_folder_artifacts + name_artifact_model

# load model
with open(path_artifact_model, 'rb') as artifact:
    model = pickle.load(artifact)

model

## RUN OPTIMIZATION

### 0. Load transversal params - sets of optimization model
Transversal all codes, not only this code. For example order in features in the data.

Save the sets of optimization model as pandas index

In [ ]:
list_regions = ['Great_Lakes',
                'Midsouth',
                'Northeast',
                'Northern_New_England',
                'Plains',
                'SouthCentral',
                'Southeast',
                'West']
regions = list_regions

regions

In [ ]:
# generate a pandas index with the values of the regions. This works as sets of optimization model
index_regions = pd.Index(regions)

### 1. Create guroby optimization model
Documentation: https://www.gurobi.com/documentation/current/refman/py_model.html

In [ ]:
# env = gp.Env(params=params)

#Create the model within the Gurobi environment
m = gp.Model(name = "Avocado_Price_Allocation")

### 2. Upper bounds and lower bounds of decision variables
Values that are boundss in decision variables. In gurobi the upper and lower boundss could be defined in the same moment that variables are created and not are defined as restrictions explicitly 

- $a_{min},a_{max}$: minimum and maximum price ($\$$) per avocado (price is a input of machine learning model)
- $b^r_{min},b^r_{max}$: minimum and maximum number of avocados allocated to region $r$

In [ ]:
# a_min, a_max: min and max price of product A
a_min = 0
a_max = 2


# b_min(r), b_max(r): min and max historical products send to each region (value get from historical data)
b_min = data_min_delivery
b_max = data_max_delivery

### 3. Input parameters of optimization model
##### That are not decision variables either parameters of machine learning model)

**Set**
- $r$ : will be used to denote each region


**Parameters Optimization Model**
- $B$: available avocados to be distributed across the regions.Total amount of avocado supply

- $c_{waste}$: cost ($\$$) per wasted avocado

- $c^r_{transport}$: cost ($\$$) of transporting a avocado to region $r$

In [ ]:
# B: supply product
B = 30 


# c_waste: cost of waste product
c_waste = 0.1


# c_transport(r): cost transport for each region
c_transport = pd.Series(
    {
        "Great_Lakes": 0.3,
        "Midsouth": 0.1,
        "Northeast": 0.4,
        "Northern_New_England": 0.5,
        "SouthCentral": 0.3,
        "Southeast": 0.2,
        "West": 0.2,
        "Plains": 0.2,
    }, name='transport_cost')
c_transport = c_transport.loc[regions]

### 4. Features input machine learning model fixed (that are not decision variables or parameters in optimization model)
Define the features that are inputs of machine learning model that are not decision variables of optimization model (so this values doesn't change). And also, this features that are not parameters of optimization model, so this values are not used in the restrictions

In [ ]:
# seasonality: 1 if it is the peak season; 0 if isn't
peak_or_not = 0
peak_or_not

In [ ]:
# list of regions
regions # in this example the regions is also a feature of machine learning model - and we want to know the price for each region(r)

In [ ]:
# generate a dataframe with the "fixed" features of optimization model. 
# This is an instance of machine learning model. In this part only have the features that have fixed values for this optimization
instance_ml_model = pd.DataFrame(
    data={
        "peak": peak_or_not,
        "region": regions,
    },
    index=regions
)
instance_ml_model

### 5. Decision variables of optimization model

Let us now define the decision variables. In our model, we want to store the price and number of avocados allocated to each region. We also want variables that track how many avocados are predicted to be sold and how many are predicted to be wasted. 

- $p(r)$ the price of an avocado ($\$$) in each region. The maxium price. It is a feature of machine learning model
- $x(r)$ the number of avocados supplied to each region
- $s(r)$ the predicted number of avocados sold in each region
- $u(r)$ the predicted number of avocados unsold (wasted) in each region
- $d(r)$ the predicted demand in each region. It is the target of machine learning model (because this value change according the input, it is a decision variable)

All those variables are created using gurobipy-pandas, with the function `gppd.add_vars`. To use this function it is necessary to define:
- model: optimization model of gurobi
- index: pandas index. With this index it can defined the sets of the decision variables
- name: name of the decision variable
- Example: x = gppd.add_vars(model, index, name="x")

In [ ]:
# p(r): price. feature of machine learning model
p = gppd.add_vars(m, index_regions, name = "price", lb = a_min, ub = a_max) # bounds prices


# x(r): supply
x = gppd.add_vars(m, index_regions, name = "x", lb = b_min, ub= b_max) # bounds supply - using historical data


# s(r): solds given a certain price
s = gppd.add_vars(m, index_regions, name = "s")


# u(r): inventary. units not sold. waste.
u = gppd.add_vars(m, index_regions, name = "w") 


# d(r): demand. output of machine learning model
d = gppd.add_vars(m, index_regions, lb = -gp.GRB.INFINITY, name = "demand") # BY DEFULT LOWER BOUND IS ZERO


#### model update - compile decision variables
m.update()

In [ ]:
### show all decision variables - debugging problems - validate after compile decision varaibles
p.gppd.VarName  # see name
p.gppd.ub # upper bound
p.gppd.lb # lowe bound

#p.gbpd.X # see value of decision variable - only works after optimization

### 6. Constraints (constraints that are not generated by a ml model)

#### 6.1 Add the Supply Constraint
Make sure that the total number of avocados supplied is equal to $B$
\begin{align*} \sum_{r} x_r &= B \end{align*}

In [ ]:
m.addConstr(x.sum() == B)
m.update()
# addConstr write the constraint implicitly

#### 6.2 Add Constraints That Define Sales Quantity
The sales quantity is the minimum of the allocated quantity and the predicted demand, i.e., $s_r = \min \{x_r,d_r(p_r)\}$ This relationship can be modeled by the following two constraints for each region $r$.

\begin{align*} s_r &\leq x_r  \\
s_r &\leq d(p_r,r) \end{align*}

In [ ]:
gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, x) # for each region (8 constraints)
gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, d)
m.update()
# add_constrs: write the constraint with left side, "operator", right side

In [ ]:
m

#### 6.3 Add the Wastage Constraints
Define the predicted unsold number of avocados in each region, given by the supplied quantity that is not sold. For each region $r$.

\begin{align*} u_r &= x_r - s_r \end{align*}

In [ ]:
gppd.add_constrs(m, u, gp.GRB.EQUAL, x - s)
m.update()

In [ ]:
m

### 7. Add constraints that are machine learning models

#### 7.1 predict demand - generate instance dataframe

First, we create our full input for the predictor constraint. We concatenate the `p` variables and the fixed features

In [ ]:
## generate a pandas instance for machine learning model

m_instance_ml_model = pd.concat([instance_ml_model, p], axis=1) # generate instance for optimization model
list_features = ['region', 'peak', 'price'] # list of the order of the features used to train the ml model
m_instance_ml_model = m_instance_ml_model[list_features] # sort instance with the correct order
m_instance_ml_model

#### 7.2 predict demand - generate machine learning constraint
Call
[add_predictor_constr](https://gurobi-machinelearning.readthedocs.io/en/stable/auto_generated/gurobi_ml.add_predictor_constr.html)
to insert the constraints linking the features and the demand into the model `m`.

It is important that you keep the columns in the order above, otherwise you will see an error. The columns must be in the same order as the training data.

Obs: to add this constraints the way is little different and it is not neccesary call model.update()

**Documentation - parameters**

- gp_model (gurobipy model) – The gurobipy model where the predictor should be inserted.

- predictor – The predictor to insert.

- input_vars (mvar_array_like) – Decision variables used as input for predictor in gp_model.

- output_vars (mvar_array_like, optional) – Decision variables used as output for predictor in gp_model.

In [ ]:
stop
AGREGAR RESTRICCIONES DE FORMA INDIVIDUAL - PASAR COMO INSTANCIA UNA SOLA REGION Y HACER ESO CON CADA UNA DE LAS REGIONES. LA IDEA ES 
PASAR DIFERENTES ELEMENTOS DEL CONJUNTO REGIONS Y NO PASAR TODOS JUNTOS

In [ ]:
# agregar restriccion de forma individual - filtrando el subconjunto del set "regions"
regions

In [ ]:
region = 'Great_Lakes'
pred_Great_Lakes = add_predictor_constr(gp_model = m, 
                                        predictor = model, 
                                        input_vars = m_instance_ml_model.loc[[region]], 
                                        output_vars = d[[region]],
                                        name = f'model_predict_{region}'
                                       )

In [ ]:
region = 'Midsouth'
pred_Midsouth = add_predictor_constr(gp_model = m, 
                                     predictor = model, 
                                     input_vars = m_instance_ml_model.loc[[region]], 
                                     output_vars = d[[region]],
                                     name = f'model_predict_{region}'
                                    )

In [ ]:
region = 'Northeast'
pred_Northeast = add_predictor_constr(gp_model = m, 
                                     predictor = model, 
                                     input_vars = m_instance_ml_model.loc[[region]], 
                                     output_vars = d[[region]],
                                     name = f'model_predict_{region}'
                                    )

In [ ]:
region = 'Northern_New_England'
pred_Northern_New_England = add_predictor_constr(gp_model = m, 
                                     predictor = model, 
                                     input_vars = m_instance_ml_model.loc[[region]], 
                                     output_vars = d[[region]],
                                     name = f'model_predict_{region}'
                                    )

In [ ]:
region = 'Plains'
pred_Plains = add_predictor_constr(gp_model = m, 
                                     predictor = model, 
                                     input_vars = m_instance_ml_model.loc[[region]], 
                                     output_vars = d[[region]],
                                     name = f'model_predict_{region}'
                                    )

In [ ]:
region = 'SouthCentral'
pred_SouthCentral = add_predictor_constr(gp_model = m, 
                                     predictor = model, 
                                     input_vars = m_instance_ml_model.loc[[region]], 
                                     output_vars = d[[region]],
                                     name = f'model_predict_{region}'
                                    )

In [ ]:
region = 'Southeast'
pred_Southeast = add_predictor_constr(gp_model = m, 
                                     predictor = model, 
                                     input_vars = m_instance_ml_model.loc[[region]], 
                                     output_vars = d[[region]],
                                     name = f'model_predict_{region}'
                                    )

In [ ]:
region = 'West'
pred_West = add_predictor_constr(gp_model = m, 
                                     predictor = model, 
                                     input_vars = m_instance_ml_model.loc[[region]], 
                                     output_vars = d[[region]],
                                     name = f'model_predict_{region}'
                                    )

In [ ]:
# ## add model to predict the demand for each region with the SAME MODEL
# pred_constr = add_predictor_constr(gp_model = m, 
#                                    predictor = model, 
#                                    input_vars = m_instance_ml_model, 
#                                    output_vars = d
#                                   )

In [ ]:
pred_West.print_stats()

In [ ]:
m

### 8. Define Objetive Function
The goal is to maximize the **net revenue**, which is the product of price and quantity, minus costs over all regions. This model assumes the purchase costs are fixed (since the amount $B$ is fixed) and are therefore not incorporated.

\begin{align} \textrm{maximize} &  \sum_{r}  (p_r * s_r - c_{waste} * u_r -
c^r_{transport} * x_r)& \end{align}

In [ ]:
m.setObjective((p * s).sum() - c_waste * u.sum() - (c_transport * x).sum(),
               gp.GRB.MAXIMIZE)

### 9. Solve optimization problem
The objective is **quadratic** since we take the product of price and the predicted sales, both of which are variables. Maximizing a quadratic
term is said to be **non-convex**, and we specify this by setting the value of the [Gurobi NonConvex
parameter](https://www.gurobi.com/documentation/10.0/refman/nonconvex.html) to be $2$.

#### 9.1 Solve optimization problem

In [ ]:
# solve cuadratic problems
m.Params.NonConvex = 2

In [ ]:
# solve
m.optimize()

#### 9.2 Save optimal values in a dataframe
To get the optimal values of decision variables it is neccesary call "var.gppd.X"

In [ ]:
# create dataframe with index
solution = pd.DataFrame(index=index_regions)

# save optimal values
solution["Price"] = p.gppd.X
solution["Historical_Max_Price"] = data_historical_max_price  # this is informative value get from historical data
solution["Allocated"] = x.gppd.X
solution["Sold"] = s.gppd.X
solution["Wasted"] = u.gppd.X
solution["Pred_demand"] = d.gppd.X

# round values
solution = solution.round(3)


# get value objetive function
opt_revenue = m.ObjVal

In [ ]:
# show value objetive function
print("\n The optimal net revenue: $%f million" % opt_revenue)

In [ ]:
# show value decision variables
solution

We can also check the error in the estimate of the Gurobi solution for the regression model.

In [ ]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_Southeast.get_error())
    )
)

In [ ]:
m

In [ ]:
print(m.numintvars)

In [ ]:
# get all variables
m.getVars()